In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [1]:
import re
import os
from tqdm import tqdm
import nltk
import pickle
import re
from pprint import pprint


In [2]:
txt = open('./data_tmp/train(12809).txt', 'r', encoding='utf-8').read()
txt = txt.split('\n\n')
train_input = []
train_output = []
for para in tqdm(txt):
    sentences = para.split('\n')
    st1 = sentences[1]
    st2 = sentences[2]
    train_input.append(st1)
    train_output.append(st2)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12801/12801 [00:00<00:00, 832500.47it/s]


In [3]:
def to_sentence(sentence, cudic):
    sentence = sentence[2:]
    text = re.sub('\[[^\[\]]*\]', '', sentence)
    pairs = re.findall('[^\[\] ]+\[[^\[\]]+\]', sentence)
    for pair in pairs:
        pair = re.split('[\[\]]', pair)
        cudic[pair[0]] = pair[1]
    words = nltk.word_tokenize(text)
    for wid, word in enumerate(words):
        if word in cudic.keys():
            words[wid] = cudic[word]
    return ' '.join(words), cudic

In [4]:
txt = open('./data_tmp/test(2030).txt', 'r', encoding='utf-8').read()
txt = txt.split('\n\n')
test_input = []
test_output = []
cudics = []
for para in tqdm(txt):
    sentences = para.split('\n')
    st1 = sentences[1]
    st2 = sentences[2]
    cudic = {}
    st_, cu = to_sentence(st1,cudic)
    cudic.update(cu)
    test_input.append(st_)
    st_, cu = to_sentence(st2,cudic)
    cudic.update(cu)
    test_output.append(st_)
    cudics.append(cudic)

                

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2031/2031 [00:00<00:00, 3398.14it/s]


In [5]:
pickle.dump(cudics,open('./data_tmp/test_cudics.pkl','wb'))
pickle.dump(test_output,open('./data_tmp/test_tars.pkl','wb'))


In [6]:
print(train_input[:5])
print(train_output[:5])
print(test_input[:5])
print(test_output[:5])
print(cudics[:5])


['Stage 2 break in skin : On sacrum and scrotum .', 'Labs were significant for d-dimer 6218 , troponin 0.05 , creatinine 1.4 , Bicarbonate 20 and white blood cells 11.3 .', 'They drove down from Country 6607 to Hospital1 18 Emergency Department for evaluation .', 'Serum toxicology test were negative for benzodiazepenes , barbituates and tricyclic antidepressants .', 'Patient was admitted to the Neurosurgery service after a study revealed a break in her shunt catheter at the level of her neck .']
['Stage 2 break in skin : On sacrum and scrotum .', 'Labs were significant for d-dimer 6218 , troponin 0.05 , creatinine 1.4 , Bicarbonate 20 and white blood cells 11.3 .', 'They drove down from Country 6607 to Hospital1 18 Emergency Department for evaluation .', 'Serum toxicology test were negative for benzodiazepenes , barbituates and tricyclic antidepressants .', 'Patient was admitted to the Neurosurgery service after a study revealed a break in your shunt catheter at the level of your neck 

In [7]:
def get_mask(src_words, tar_words):
    temp = []
    tar_ptr = 0
    for i, word in enumerate(src_words):
        if i+1 < len(src_words) and src_words[i+1] in tar_words[tar_ptr:]:
            j = tar_words.index(src_words[i+1], tar_ptr)+1
        else:
            j = len(tar_words)
        if word not in tar_words[tar_ptr:j]:
            temp.append(1)
        else:
            temp.append(0)
            tar_ptr = tar_words.index(word, tar_ptr) + 1
    return temp


In [8]:
for i in tqdm(range(len(train_input))):
    st1 = train_input[i]
    st2 = train_output[i]
    words1 = nltk.word_tokenize(st1)
    words2 = nltk.word_tokenize(st2)
    masks=get_mask(words1, words2)
    new_words1 = []
    new_words2 = []
    new_words1.append('[CLS]')
    new_words2.append('[CLS]')
    new_words2.extend(words2)
    flag = 0
    for j, mask in enumerate(masks):
        if mask == 0 and flag == 1:
            new_words1.append('[unused2]')
            flag = 0
        elif mask == 1 and flag == 0:
            new_words1.append('[unused2]')
            flag = 1
        new_words1.append(words1[j])
    new_words1.append('[SEP]')
    new_words2.append('[SEP]')
    train_input[i] = ' '.join(new_words1)
    train_output[i] = ' '.join(new_words2)
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12801/12801 [00:03<00:00, 3258.64it/s]


In [11]:
for i in tqdm(range(len(test_input))):
    st1 = test_input[i]
    st2 = test_output[i]
    words1 = nltk.word_tokenize(st1)
    words2 = nltk.word_tokenize(st2)
    masks=get_mask(words1, words2)
    new_words1 = []
    new_words2 = []
    new_words1.append('[CLS]')
    new_words2.append('[CLS]')
    new_words2.extend(words2)
    flag = 0
    for j, mask in enumerate(masks):
        if mask == 0 and flag == 1:
            new_words1.append('[unused2]')
            flag = 0
        elif mask == 1 and flag == 0:
            new_words1.append('[unused2]')
            flag = 1
        new_words1.append(words1[j])
    new_words1.append('[SEP]')
    new_words2.append('[SEP]')
    test_input[i] = ' '.join(new_words1)
    test_output[i] = ' '.join(new_words2)
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2031/2031 [00:00<00:00, 3551.39it/s]


In [20]:

for i,(u,v) in enumerate(zip(test_input,test_output)):
    if 'patient is a 89 year old' in u:
        print(u)
        print(v)
        print(i)

[CLS] patient is a 89 year old male with [unused2] sick sinus syndrome status post [unused2] pacer , admitted to Neurosurgery with [unused2] subdural hematoma [unused2] with acute respiratory distress/ wheezing on the floor . [SEP]
[CLS] patient is a 89 year old male with irregular heart rhythms after pacer , admitted to Neurosurgery with brain blood collects with acute respiratory distress/ wheezing on the floor . [SEP]
8


In [13]:
with open("./data_tmp/train_input.pkl", 'wb') as f:
    pickle.dump(train_input,f)
with open("./data_tmp/train_output.pkl", 'wb') as f:
    pickle.dump(train_output,f)
with open("./data_tmp/test_input.pkl", 'wb') as f:
    pickle.dump(test_input,f)
with open("./data_tmp/test_output.pkl", 'wb') as f:
    pickle.dump(test_output,f)



In [21]:
from transformers import BertTokenizer
from importlib import import_module
import random
from keras.preprocessing.sequence import pad_sequences
model_name = 'bert'
x = import_module('models.' + model_name)
config = x.Config(16)
tokenizer = config.tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = False)

def tokenize(input_data, output_data):
    ids = []
    labels = []
    masks = []
    for i in tqdm(range(len(input_data))):
#         encoded_dict = tokenizer.encode_plus(
#                                 input_data[i],                      # Sentence to encode.
#                                 add_special_tokens = False, # Add '[CLS]' and '[SEP]'
#                                 max_length = 64,           # Pad & truncate all sentences.
#                                 pad_to_max_length = True,
#                                 return_attention_mask = True,   # Construct attn. masks.
#                                 return_tensors = 'pt',     # Return pytorch tensors.
#                            )
#         words = nltk.word_tokenize(input_data[i])
#         tokenized_sentence = []
#         for word in words:
#             tokenized_word = tokenizer.tokenize(word)
#             tokenized_sentence.extend(tokenized_word)
#         ids.append(tokenized_sentence)
        
#         words = nltk.word_tokenize(output_data[i])
#         tokenized_sentence = []
#         for word in words:
#             tokenized_word = tokenizer.tokenize(word)
#             tokenized_sentence.extend(tokenized_word)
#         labels.append(tokenized_sentence)
        words = tokenizer.tokenize(input_data[i])
        ids.append(words)
        words = tokenizer.tokenize(output_data[i])
        labels.append(words)
        
        
    
    ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in ids],
                          maxlen=64, dtype="long", value=0,
                              truncating="post", padding="post")
    labels = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in labels],
                          maxlen=64, dtype="long", value=0,
                              truncating="post", padding="post")
    
        
    return ids, labels
    
#         ids.append(encoded_dict['input_ids'])
#         masks.append(encoded_dict['attention_mask'])
#         encoded_dict = tokenizer.encode_plus(
#                                 output_data[i],add_special_tokens = False, max_length = 64, pad_to_max_length = True, return_attention_mask = True,
#                                 return_tensors = 'pt',)
#         labels.append(encoded_dict['input_ids'])
#     return ids, labels, masks
    
# train_ids, train_labels, train_mask = tokenize(train_input, train_output)
# test_ids, test_labels, test_mask = tokenize(test_input, test_output)
train_src, train_tar = tokenize(train_input, train_output)
test_src, test_tar= tokenize(test_input, test_output)
train_src_masks = [[float(i != 0.0) for i in ii] for ii in train_src]
train_tar_masks = [[float(i != 0.0) for i in ii] for ii in train_tar]
test_src_masks = [[float(i != 0.0) for i in ii] for ii in test_src]
test_tar_masks = [[float(i != 0.0) for i in ii] for ii in test_tar]



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2031/2031 [00:00<00:00, 2097.72it/s]


In [22]:
pickle.dump(train_src, open("./data_tmp/train_src.pkl",'wb'))
pickle.dump(train_tar, open("./data_tmp/train_tar.pkl",'wb'))
pickle.dump(train_src_masks, open("./data_tmp/train_src_masks.pkl",'wb'))
pickle.dump(train_tar_masks, open("./data_tmp/train_tar_masks.pkl",'wb'))


pickle.dump(test_src, open("./data_tmp/test_src.pkl",'wb'))
pickle.dump(test_tar, open("./data_tmp/test_tar.pkl",'wb'))
pickle.dump(test_src_masks, open("./data_tmp/test_src_masks.pkl",'wb'))
pickle.dump(test_tar_masks, open("./data_tmp/test_tar_masks.pkl",'wb'))



In [23]:
import numpy as np
print(np.max(train_src),np.min(train_src))
print(np.max(test_src),np.min(test_src))
print(np.max(train_tar),np.min(train_tar))
print(np.max(test_tar),np.min(test_tar))

30519 0
30511 0
30519 0
30511 0


In [ ]:

train_ids = torch.tensor(train_ids).to(torch.int64)
test_ids = torch.tensor(test_ids).to(torch.int64)
train_masks = torch.tensor(train_masks).to(torch.int64)
test_masks = torch.tensor(test_masks).to(torch.int64)
train_labels = torch.tensor(train_labels).to(torch.int64)
test_labels = torch.tensor(test_labels).to(torch.int64)

In [ ]:
# import numpy as np
# st = set()
# for u in train_labels:
#     L = np.array(u)[0]
#     for v in L:
#         st.add(v)
# print(len(st))
# for u in test_labels:
#     L = np.array(u)[0]
#     for v in L:
#         st.add(v)
# print(len(st))



In [ ]:
# train_ids = torch.cat(train_ids, dim=0)
# train_mask = torch.cat(train_mask, dim=0)
# train_labels = torch.cat(train_labels, dim=0)

# test_ids = torch.cat(test_ids, dim=0)
# test_mask = torch.cat(test_mask, dim=0)
# test_labels = torch.cat(test_labels, dim=0)


In [ ]:
print(train_labels.max())
print(train_labels.min())
print(train_ids.max())
print(train_ids.min())

print(test_labels.max())
print(test_labels.min())
print(test_ids.max())
print(test_ids.min())



In [ ]:
MAX_LEN = 64
bs = 16

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
print(torch.__version__)

train_data = TensorDataset(train_ids, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(test_ids, test_masks, test_labels)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
import transformers
from transformers import BertForTokenClassification, AdamW

transformers.__version__

In [ ]:
model = BertForTokenClassification.from_pretrained("bert-base-cased",num_labels=3,
                                                    output_attentions = False,
                                                    output_hidden_states = False)


In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=5e-5,
    eps=1e-8
)

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 3
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
## Store the average loss after each epoch so we can plot them.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
print(device)

n_gpu = torch.cuda.device_count()
print(n_gpu)
model = model.to(device)

In [ ]:

loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        

        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
#         b_input_ids = torch.tensor(b_input_ids).to(torch.int64)
#         b_input_mask = torch.tensor(b_input_mask).to(torch.int64)
#         b_labels = torch.tensor(b_labels).to(torch.int64)

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PADT"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PADT"]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    print()
